In [2]:
import matplotlib.pyplot as plt

from Utils.UtilsViz import *

In [3]:
# What are the data files provided?
data_path = os.path.join(os.getcwd(), "datasets")
projects_data = pd.read_csv(os.path.join(data_path, "projects.csv"), nrows=100)

In [3]:
print(" Projects data --> ", projects_data.shape)
print_df_cols(projects_data)

 Projects data -->  (664098, 35)
Columns : 
	 projectid
	 teacher_acctid
	 schoolid
	 school_ncesid
	 school_latitude
	 school_longitude
	 school_city
	 school_state
	 school_zip
	 school_metro
	 school_district
	 school_county
	 school_charter
	 school_magnet
	 school_year_round
	 school_nlns
	 school_kipp
	 school_charter_ready_promise
	 teacher_prefix
	 teacher_teach_for_america
	 teacher_ny_teaching_fellow
	 primary_focus_subject
	 primary_focus_area
	 secondary_focus_subject
	 secondary_focus_area
	 resource_type
	 poverty_level
	 grade_level
	 fulfillment_labor_materials
	 total_price_excluding_optional_support
	 total_price_including_optional_support
	 students_reached
	 eligible_double_your_impact_match
	 eligible_almost_home_match
	 date_posted



Let's explore the Project dataset

In [ ]:
project_list = projects_data["projectid"].unique()
print("Total number of project IDs :", project_list.shape[0])

school_list =  projects_data["schoolid"].unique()
print("Total number of School IDs :", school_list.shape[0])

school_county_list =  projects_data["school_county"].unique()
print("Total number of School Counties :", school_county_list.shape[0])

resource_type_list =  projects_data["resource_type"].unique()
print("Total number of Resource Types :", resource_type_list.shape[0], resource_type_list)

poverty_level_list =  projects_data["poverty_level"].unique()
print("Total number of Poverty Levels :", poverty_level_list.shape[0], poverty_level_list)

grade_level_list =  projects_data["grade_level"].unique()
print("Total number of Grade Levels :", grade_level_list.shape[0], grade_level_list)

In [11]:
# First we declare some helper variables and get the GEOJSON data for the counties.
# Get the path of the GeoJSON file for the chiorpleth map
_path = os.getcwd()
county_geojson_fpath = "D:/Dev/Sources/Projects/GitProjects/KDDCup_2014/res/gz_2010_us_050_00_500k.json"
# NOTE : the key to look for in the json file is 'feature.properties.NAME'
US_coord = [37.0902, -102]

In [13]:
county_schoolconc_data = projects_data[["school_county", "schoolid"]].groupby(by="school_county", as_index=False).count()
th_scale = get_th_scale(data=county_schoolconc_data, col="schoolid", n_steps=4)
# Draw the map and visualise
map1 = draw_choropleth_map(json_path=county_geojson_fpath, json_key='feature.properties.NAME',
                           threshold_scale=th_scale, start_coord=US_coord,
                          data=county_schoolconc_data, data_cols=list(county_schoolconc_data.columns),
                          legend_name="Number of schools per county")
open_map_in_browser(geomap=map1, path=os.path.join("D:/Dev/Sources/Projects/GitProjects/KDDCup_2014/res/", 'school_conc.html'))

In [20]:
# Is there a correlation between students reached in a county and number of schools in a county?
county_stdreach_data = projects_data[["school_county", "students_reached"]].groupby(by="school_county", as_index=False).sum()
county_sc_sr_data = pd.merge(left=county_schoolconc_data, right=county_stdreach_data, on="school_county")
print(np.round(county_sc_sr_data.corr(), 2))
# Very high correlation there. It's safe to say that we do not need to make a separate choropleth map for students 

                  schoolid  students_reached
schoolid              1.00              0.97
students_reached      0.97              1.00


In [23]:
# Encode the categorical data poverty level 
temp_df = projects_data[["school_county", "poverty_level"]]
categories = {
    'highest poverty':3,
    'high poverty':2,
    'moderate poverty':1,
    'low poverty':0
}
def set_category(val):
    return categories[val]
temp_df["poverty_level"] = temp_df.apply(lambda row: set_category(row["poverty_level"]), axis=1)

D:\Softwares\Python36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
county_schpoverty_data = temp_df.groupby(by="school_county", as_index=False).mean()
county_schpoverty_data["poverty_level"] = county_schpoverty_data["poverty_level"]*100
county_schpoverty_data["poverty_level"] = county_schpoverty_data["poverty_level"].astype(int)


In [11]:
# How many students did this program reach?
print(projects_data["students_reached"].sum())


64558408.0


In [ ]:
# What are all the commonly named columns present in each dataset? 

# all_cols = []
# all_cols.append(list(donations_data.columns))
# all_cols.append(list(resources_data.columns))
# all_cols.append(list(projects_data.columns))
# common_cols = find_common_cols(all_cols)
# print(common_cols)

# This really works well with the metadata provided. We can join these datasets with the project ID to make one consolidated dataset

# d_p_data = pd.merge(donations_data, projects_data, how='inner', on="projectid")

# consolidated_data = pd.merge(d_p_data, resources_data, how='inner', on="projectid")

# consolidated_data.shape

# donations_data.head()


# donations_data.projectid.value_counts(ascending=False)

# # Cleaning

# # Find rows with NaNs
# nan_row_cnt = donations_data.dropna().shape[0]
# print("Number of rows with NaNs in them :", nan_row_cnt)
# print("\t in percentage :", nan_row_cnt / donations_data.shape[0])
# # Clearly dropping them all blindly is NOT the best solution. 



# # What about the categorical variables?
# cities = donations_data["donor_city"].unique()
# print("Number of cities : ", cities.shape[0])

# states = donations_data["donor_state"].unique()
# print("Number of states : ", states.shape[0])